# Fonction Python et symbolique

In [1]:
import openturns as ot

### Exercice 1 : une fonction avec 4 entrées

On considère un nouveau modèle, avec une nouvelle variable de sortie
Y3 et une nouvelle variable d’entrée X4 :
$$
\begin{eqnarray}
Y_1 &=& X_1 + X_2 + X_3 \\
Y_2 &=& X_1 - X_2 X_3 \\
Y_3 &=& 2 X_1 + 3 X_2 + 4 X_4
\end{eqnarray}
$$

**Questions**
* Modifier la fonction Python pour simuler le nouveau modèle.
* Ajouter une nouvelle variable X4 de loi normale standard dans le modèle probabiliste.
* Estimer la moyenne de la sortie par Monte-Carlo simple.

### Solution de l'exercice 1 :  une fonction avec 4 entrées


In [2]:
def mySimulator(x):
    y0 = x[0] + x[1] + x[2]
    y1 = x[0] - x[1] * x[2]
    y2 = 2 * x[0] + 3 * x[1] + 4 * x[3]
    y = [y0, y1, y2]
    return y

In [3]:
myfunction = ot.PythonFunction(4, 3, mySimulator)
# Create the marginal distributions
X0 = ot.Normal(0.0, 1.0)
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)
X3 = ot.Normal(0.0, 1.0)
# Create the input probability distribution
inputDistribution = ot.ComposedDistribution((X0, X1, X2, X3))
# Create the input random vector
inputRandomVector = ot.RandomVector(inputDistribution)
# Create the output variable of interest
outputVariableOfInterest = ot.CompositeRandomVector(myfunction, inputRandomVector)

In [4]:
# Probabilistic Study: central dispersion
montecarlosize = 10000
# Start the simulations
outputSample = outputVariableOfInterest.getSample(montecarlosize)
# Get the empirical mean and standard deviations
outputSample.computeMean()

class=Point name=Unnamed dimension=3 values=[-0.0166778,-0.0123527,-0.0277928]

### Exercice 2 : gradient d'une fonction Python

OT peut calculer la dérivée approchée d’une fonction Python par différences finies. On peut paramétrer la formule de différence utilisée, ainsi que le pas de différenciation de cette formule. De plus, lorsque la matrice Jacobienne est implémentée dans une fonction Python, on peut transmettre cette fonction à OpenTURNS pour qu'il l'utilise.

**Questions**
* Définir la fonction `myfunction` comme dans le sujet, c'est à dire avec 3 entrées et 2 sorties.
* Utiliser la méthode `gradient` de l’objet `myfunction` pour évaluer le
gradient G'(x) au point d’entrée X = (1, 2, 3).
* Utiliser la méthode `hessian` de l’objet `myfunction` pour évaluer la
matrice Hessienne de G.

* Utiliser les instructions suivantes pour configurer un gradient calculé
par une formule de différences finies décentrée, avec un pas $h = 10^{−2}$.

```
myfunctionImpl = myfunction.getEvaluation()
h = 1.e-2
myGradient = ot.NonCenteredFiniteDifferenceGradient(h, myfunctionImpl)
myfunction.setGradient(myGradient)
```

* Evaluer à nouveau le gradient avec la méthode gradient et comparer avec le résultat précédent.
* On peut transmettre à OT une fonction Python qui évalue le gradient. Pour cela on peut utiliser la séquence d'appel :
```
myfunction = ot.PythonFunction(nbInputs, nbOutputs, mySimulator, gradient=mySimulatorGradient)
```
où `mySimulatorGradient` est une fonction Python qui évalue le gradient.
Calculez à la main des dérivées partielles de la fonction G associée à l'exemple fil rouge. 
Puis définissez la fonction `mySimulatorGradient` qui évalue la matrice Jacobienne. Puisqu'il y a trois variables d'entrée, la liste renvoyée par `mySimulatorGradient` doit contenir trois éléments. Chaque élément doit contenir une sous-liste de taille 2 contenant les dérivées de chaque sortie. Enfin, construisez la fonction associée avec l'option `gradient`. 

### Solution de l'exercice 2 : gradient d'une fonction Python

In [5]:
def mySimulator(x):
    y0 = x[0] + x[1] + x[2]
    y1 = x[0] - x[1] * x[2]
    y = [y0, y1]
    return y

In [6]:
inputDim = 3
outputDim = 2
myfunction = ot.PythonFunction(inputDim, outputDim, mySimulator)

Evaluer le gradient.

In [7]:
d = myfunction.gradient([1.0, 2.0, 3.0])
print("type(d)=", type(d))  # OT Matrix
print("Gradient par DF=")
d

type(d)= <class 'openturns.typ.Matrix'>
Gradient par DF=


class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=2 values=[1,1,1,1,-3,-2]

Evaluer la hessienne.

In [8]:
dd = myfunction.hessian([1.0, 2.0, 3.0])
print("type(dd)=", type(dd))  # OT SymmetricTensor
print("Hessienne=")
dd

type(dd)= <class 'openturns.typ.SymmetricTensor'>
Hessienne=


class=SymmetricTensor implementation=class=TensorImplementation name=Unnamed rows=3 columns=3 sheets=2 values=[0,2.22045e-08,0,0,0,2.22045e-08,0,0,0,0,0,0,0,0,-1,0,0,0]

Configurer la formule de différences finies du gradient. 

In [9]:
myfunctionImpl = myfunction.getEvaluation()
myGradient = ot.NonCenteredFiniteDifferenceGradient(1.0e-2, myfunctionImpl)
myfunction.setGradient(myGradient)

In [10]:
d = myfunction.gradient([1, 2, 3])
print("Gradient par DF non centrée=")
d

Gradient par DF non centrée=


class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=2 values=[1,1,1,1,-3,-2]

Configurer le gradient avec une fonction Python. 

In [11]:
def mySimulatorGradient(x):
    dyx0 = [1.0, 1.0]
    dyx1 = [1.0, -x[2]]
    dyx2 = [1.0, -x[1]]
    y = [dyx0, dyx1, dyx2]
    return y

In [12]:
myfunction = ot.PythonFunction(3, 2, mySimulator, gradient=mySimulatorGradient)
d = myfunction.gradient([1.0, 2.0, 3.0])
print("d - Exact =")
d

d - Exact =


class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=2 values=[1,1,1,1,-3,-2]

### Exercice 3 : gestion de l'historique d'une fonction Python

**Questions**
* Observer le changement de la valeur de retour de `isHistoryEnabled`
* Quelles sont les méthodes qui permettent de récupérer les historiques des entrées et des sorties ?
* Comment avoir le nombre d’appels à la fonction ?
* Utiliser la méthode `clearHistory` et vérifier que l'historique est vide après cet appel.

### Solution de l'exercice 3 : gestion de l'historique d'une fonction Python

In [13]:
def mySimulator(x):
    y0 = x[0] + x[1] + x[2]
    y1 = x[0] - x[1] * x[2]
    y = [y0, y1]
    return y

In [14]:
myfunction = ot.PythonFunction(3, 2, mySimulator)
myfunction = ot.MemoizeFunction(myfunction)

In [15]:
# Create the marginal distributions
X0 = ot.Normal(0.0, 1.0)
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)

In [16]:
# Create the input probability distribution
inputDistribution = ot.ComposedDistribution((X0, X1, X2))
# Create the input random vector
inputRandomVector = ot.RandomVector(inputDistribution)
# Create the output variable of interest
outputVariableOfInterest = ot.CompositeRandomVector(myfunction, inputRandomVector)
# Probabilistic Study: central dispersion
montecarlosize = 20
outputSample = outputVariableOfInterest.getSample(montecarlosize)

In [17]:
# Get the history
inputs = myfunction.getInputHistory()
print("inputs")
print(inputs)
outputs = myfunction.getOutputHistory()
print("outputs")
print(outputs)
# Nombre d'appels à la fonction G
nGEvals = inputs.getSize()
print("nGEvals = %d" % (nGEvals))

inputs
 0 : [  1.09671     0.0519288   0.986608   ]
 1 : [  0.482945   -0.769778    0.163746   ]
 2 : [ -1.45047    -1.2523      1.0915     ]
 3 : [  0.0964844   1.51156    -0.66498    ]
 4 : [  0.762276   -1.14365     0.271192   ]
 5 : [ -0.875386   -0.188975   -0.0047933  ]
 6 : [  0.753207   -1.01182    -0.170364   ]
 7 : [ -1.62312    -0.386589   -0.572422   ]
 8 : [ -0.451058    0.399015   -0.735141   ]
 9 : [ -0.87281     2.50848    -1.04574    ]
10 : [  2.06075     0.442079    0.821644   ]
11 : [  0.0711821   2.22299     1.83813    ]
12 : [ -1.5925     -0.329925    0.398789   ]
13 : [ -0.296501   -0.954536   -0.619029   ]
14 : [  0.00985945 -0.958055   -0.0859105  ]
15 : [ -0.383314   -0.642792   -0.469882   ]
16 : [  0.190781    1.03291     0.93799    ]
17 : [ -0.760095   -0.940847    0.24978    ]
18 : [  0.351863    0.487823   -0.167711   ]
19 : [ -0.891095    1.76734    -1.1513     ]
outputs
 0 : [  2.13524    1.04547   ]
 1 : [ -0.123087   0.608993  ]
 2 : [ -1.61126   -0.08

In [18]:
# Clear the history
myfunction.clearHistory()

In [19]:
# See how the history is now empty
print("After clearHistory:")
myfunction.getOutputHistory()

After clearHistory:


class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=0 dimension=2 data=[]

### Exercice 4 : fonction symbolique avec 4 entrées

On considère le modèle :
$$
\begin{eqnarray}
Y_1 &=& X_1 + X_2 + X_3 \\
Y_2 &=& X_1 - X_2 X_3 \\
Y_3 &=& 2 X_1 + 3 X_2 + 4 X_4
\end{eqnarray}
$$

**Questions**
* Créer une fonction symbolique pour créer ce nouveau modèle.
* Evaluer la sortie du modèle au point $X=(1,2,3,4)^T$.
* Estimer la moyenne de la sortie par Monte-Carlo simple.

### Solution de l'exercice 4 : fonction symbolique avec 4 entrées

In [20]:
myfunctionSymbolic4 = ot.SymbolicFunction(
    ("x0", "x1", "x2", "x3"),
    ("x0 + x1 + x2", "x0 - x1 * x2", "2 * x0 + 3 * x1 + 4 * x3"),
)
X = ot.Point([1, 2, 3, 4])
Y = myfunctionSymbolic4(X)
Y

class=Point name=Unnamed dimension=3 values=[6,-5,24]

In [21]:
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)
X3 = ot.Normal(0.0, 1.0)
X4 = ot.Normal(0.0, 1.0)
inputDistribution = ot.ComposedDistribution((X1, X2, X3, X4))
inputRandomVector = ot.RandomVector(inputDistribution)
outputVect = ot.CompositeRandomVector(myfunctionSymbolic4, inputRandomVector)
montecarlosize = 10000
outputSample = outputVect.getSample(montecarlosize)
empiricalMean = outputSample.computeMean()
print(empiricalMean)

[-0.00501945,0.00440618,0.0636051]


### Exercice 5 : fonction symbolique avec paramètres

On considère le modèle 
$$
\begin{eqnarray}
Y_1 &=& a X_1 + b X_2 \\
Y_2 &=& c X_1 + d X_2
\end{eqnarray}
$$
où a, b, c, d sont des paramètres :
```
a = 12
b = 23
c = -34
d = 45
```

**Questions**
* Créer une fonction symbolique pour créer ce nouveau modèle en utilisant la fonction `str`.
* Evaluer la sortie du modèle au point $X=(1,2)^T$.

Note : la classe `ParametricFunction` est plus adaptée pour cela.

### Solution de l'exercice 5 : fonction symbolique avec paramètres

In [22]:
function = ot.SymbolicFunction(
    ("x0", "x1", "a", "b", "c", "d"), ("a * x0 + b * x1", "c * x0 + d * x1")
)

In [23]:
x0 = 1.0
x1 = 2.0
a = 12.0
b = 23.0
c = -34.0
d = 45.0
X = [x0, x1, a, b, c, d]
function(X)

class=Point name=Unnamed dimension=2 values=[58,56]

In [24]:
indices = [2, 3, 4, 5]
referencePoint = [a, b, c, d]
g = ot.ParametricFunction(function, indices, referencePoint)

In [25]:
X = ot.Point([x0, x1])
Y = g(X)
Y

class=Point name=Unnamed dimension=2 values=[58,56]

### Exercice 6 : gradient d'une fonction symbolique

On souhaite vérifier que OT peut calculer la dérivée formelle d’une
fonction symbolique. 

**Questions**
* Définir la fonction `myfunctionSymbolic` comme dans l’exemple fil rouge.
* Créer la variable `myGradient` contenant la dérivée exacte de la fonction. Pour cela, utiliser la méthode `getGradient` de l’objet
`myfunctionSymbolic`. 

* Qu’est-ce qui s’affiche quand on utilise l’instruction suivante ?

`
print(myGradient)
`

* On souhaite évaluer le gradient au point d’entrée suivant :
`
X = (1, 2, 3)
`
Utiliser la méthode `gradient` de l’objet `myGradient` pour évaluer G'(x).

### Solution de l'exercice 6 : gradient d'une fonction symbolique

In [26]:
myfunction = ot.SymbolicFunction(("x0", "x1", "x2"), ("x0 + x1 + x2", "x0 - x1 * x2"))
print(myfunction)
#
myGradient = myfunction.getGradient()
print(myGradient)
#
myGradient.gradient([1.0, 2.0, 3.0])

[x0,x1,x2]->[x0 + x1 + x2,x0 - x1 * x2]

| d(y0) / d(x0) = 1
| d(y0) / d(x1) = 1
| d(y0) / d(x2) = 1
| d(y1) / d(x0) = 1
| d(y1) / d(x1) = -1*x2
| d(y1) / d(x2) = -1*x1



class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=2 values=[1,1,1,1,-3,-2]

### Exercice 7 : gestion des variables intermédiaires dans une fonction symbolique

On peut définir une fonction symbolique dont l'évaluation est fondée sur des valeurs intermédiaires. Ainsi, la sortie n'est pas seulement une fonction explicite des entrées : on peut définir des résultats intermédiaires et les réutiliser dans une ou plusieurs sorties de la fonction. 

Pour cela, il faut utiliser la séquence d'appel suivante :
```
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
```
où `outputs` est une chaîne de caractères contenant l'expression à évaluer. 

Pour composer cette chaîne de caractère, on peut définir plusieurs expressions, séparées par le caractère ";". De plus, les variables intermédiaires doivent être précédées du mot-clé "`var`". 

Par exemple, dans le cas du modèle dont les entrées sont $X_1$ et $X_2$ et les sorties sont $Y_1$ et $Y_2$ :
$$
\begin{eqnarray}
T &=& X_1 X_2 \\
Y_1 &=& X_1 + T \\
Y_2 &=& X_2 - 3T
\end{eqnarray}
$$
on peut utiliser l'instruction suivante :

In [27]:
inputs = ["X1", "X2"]
outputs = ["Y1", "Y2"]
formula = "var T := X1 * X2; Y1 := X1 + T; Y2 := X2 - 3 * T"
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
myFunction([1.0, 2.0])

class=Point name=Unnamed dimension=2 values=[3,-4]

Définir le modèle dont les entrées sont $X_1$ et $X_2$ et les sorties sont $Y_1$ et $Y_2$ :
$$
\begin{eqnarray}
S &=& X_1 + X_2 \\
T &=& X_1 X_2 \\
Y_1 &=& S + T \\
Y_2 &=& ST
\end{eqnarray}
$$

### Solution de l'exercice 7 : gestion des variables intermédiaires dans une fonction symbolique

In [28]:
inputs = ["X1", "X2"]
outputs = ["Y1", "Y2"]
formula = "var S := X1 + X2; var T := X1 * X2; Y1 := S + T; Y2 := S * T"
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
myFunction([1.0, 2.0])

class=Point name=Unnamed dimension=2 values=[5,6]

## Exercice 8 : configurer le nombre de cpus

L'option `n_cpus` de la classe `PythonFunction` permet de configurer le nombre de processeurs. L'implémentation est fondée sur le module `multiprocessing`. Dans cet exercice, on cherche à observer l'effet de cette option sur la performance du calcul.

Pour observer un changement dans la performance nous nous plaçons dans la situation suivante :
- la fonction possède un grand nombre de variables d'entrées,
- la fonction est coûteuse.

Dans ce but, nous définissons la fonction suivante.

In [29]:
import math


def myHighDimSimulator(x):
    xPoint = ot.Point(x)
    dim = xPoint.getDimension()
    y0 = 0.0
    y1 = 1.0
    for i in range(dim):
        y0 = y0 + math.exp(x[i])
        y1 = y1 * math.exp(x[i])
    y = [y0, y1]
    return y

In [30]:
dim = 100
inputHighDimDistribution = ot.ComposedDistribution([ot.Normal()] * dim)
inputHighDimRandomVector = ot.RandomVector(inputHighDimDistribution)

- Utiliser le module `time` pour mesurer la performance de la fonction sans l'option `n_cpus`. Pour observer une durée de simulation significative, augmentez la taille du plan d'expériences ou le nombre de dimensions.
- De même avec l'option `n_cpus`.
- Quelle différence constatez-vous ?

## Solution de l'exercice 8 : configurer le nombre de cpus

In [31]:
import time

In [32]:
def benchMyPythonFunction(inputRandomVector, mypyfunction, sampleSize, label):
    t0 = time.time()
    outputVect = ot.CompositeRandomVector(mypyfunction, inputRandomVector)
    outputSample = outputVect.getSample(sampleSize)
    t1 = time.time()
    print("Elapsed = %.2f (s)" % (t1 - t0))

In [33]:
sampleSize = 100

In [34]:
myPyFunction1 = ot.PythonFunction(dim, 2, myHighDimSimulator)
benchMyPythonFunction(
    inputHighDimRandomVector, myPyFunction1, sampleSize, "Without n_cpus"
)

Elapsed = 0.01 (s)


In [35]:
myPyFunction2 = ot.PythonFunction(dim, 2, myHighDimSimulator, n_cpus=8)
benchMyPythonFunction(
    inputHighDimRandomVector, myPyFunction2, sampleSize, "With n_cpus"
)

Elapsed = 0.08 (s)
